In [1]:
import atoti as tt

Welcome to Atoti 0.8.10!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

Atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, you can request a trial license at https://www.atoti.io/evaluation-license-request.

You can hide this message by setting the `ATOTI_HIDE_EULA_MESSAGE` environment variable to True.


In [2]:
session = tt.Session(                            #Creare atoti session
    user_content_storage=".content",
    port=9095,
    java_options=["-Xms1G", "-Xmx10G"]
)

In [3]:
db_name = "Project1"
db_user = "postgres"
db_password = "postgres"  
db_host = "localhost"  
db_port = "5432"

In [4]:
jdbc_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}?user={db_user}&password={db_password}" #Create URL

In [5]:
jdbc_url

'jdbc:postgresql://localhost:5432/Project1?user=postgres&password=postgres'

In [6]:
fact_olympic = session.read_sql(       #Get main fact table
    "SELECT * FROM factolympic",
    keys=["year","country"],
    table_name="Olympic",
    url=jdbc_url,
)

In [7]:
dim_countries = session.read_sql(     #Get countries facttable
    "SELECT * FROM dimcountries",
    keys=["country"],
    table_name="Countries",
    url=jdbc_url)

In [8]:
dim_life = session.read_sql( #Get life expectancy dimension table
    "SELECT * FROM dimlife",
    keys=["year","country"],
    table_name="Life",
    url=jdbc_url)

In [9]:
dim_time = session.read_sql(  #Get time dimension table
    "SELECT * FROM dimtime",
    keys=["year"],
    table_name="Time",
    url=jdbc_url)

In [10]:
fact_olympic.join(dim_countries, fact_olympic["country"] == dim_countries["country"])  #Join tables based 

In [11]:
fact_olympic.join(dim_life, (fact_olympic["year"] == dim_life["year"] )& (fact_olympic["country"] == dim_life["country"]))

In [12]:
fact_olympic.join(dim_time, fact_olympic["year"] == dim_time["year"])

In [13]:
session.tables.schema

```mermaid
erDiagram
  "Countries" {
    _ String PK "country"
    _ String "continent"
  }
  "Life" {
    _ int PK "year"
    _ String PK "country"
    _ String "lifeexpectancy"
    _ String "population"
  }
  "Time" {
    _ int PK "year"
    _ String "decade"
    _ String "season"
  }
  "Olympic" {
    _ int PK "year"
    _ String PK "country"
    nullable int "bronze"
    nullable int "silver"
    nullable int "gold"
    nullable int "total"
    nullable double "depression"
    nullable double "schizophrenia"
    nullable double "bipolar"
    nullable double "eatingdisorder"
    nullable double "anxiety"
  }
  "Olympic" }o--o| "Countries" : "`country` == `country`"
  "Olympic" }o--o| "Life" : "(`year` == `year`) & (`country` == `country`)"
  "Olympic" }o--o| "Time" : "`year` == `year`"
```


In [14]:
testcube = session.create_cube(fact_olympic)  #Cube creation

In [15]:
testcube

In [16]:
hierarchies, levels, measures = testcube.hierarchies, testcube.levels, testcube.measures

In [23]:
hierarchies["Countries","continent"] = [dim_countries["continent"],dim_countries["country"]]
hierarchies["Time","time"] = [dim_time["decade"],dim_time["year"],dim_time['season']]
hierarchies["Lifexpectancy","lifeexpectancy"] = [dim_life["lifeexpectancy"],dim_life['population']]

In [18]:
del hierarchies["Olympic","country"]
del hierarchies["Olympic","year"]
del hierarchies["Time","decade"]
del hierarchies['Time','season']

In [27]:
del hierarchies['Life', 'population']
del hierarchies ['Life', 'lifeexpectancy']

In [28]:
list(levels)

[('Countries', 'continent', 'continent'),
 ('Countries', 'continent', 'country'),
 ('Time', 'time', 'decade'),
 ('Time', 'time', 'year'),
 ('Time', 'time', 'season'),
 ('Lifexpectancy', 'lifeexpectancy', 'lifeexpectancy'),
 ('Lifexpectancy', 'lifeexpectancy', 'population')]

In [29]:
measures

{'contributors.COUNT': <atoti.measure.Measure object at 0x75cdcc5565c0>, 'update.TIMESTAMP': <atoti.measure.Measure object at 0x75cdaf417070>, 'depression.MEAN': <atoti.measure.Measure object at 0x75cdaf417010>, 'depression.SUM': <atoti.measure.Measure object at 0x75cdaf416fe0>, 'bipolar.MEAN': <atoti.measure.Measure object at 0x75cdaf417130>, 'bipolar.SUM': <atoti.measure.Measure object at 0x75cdaf417100>, 'bronze.MEAN': <atoti.measure.Measure object at 0x75cdaf4170a0>, 'bronze.SUM': <atoti.measure.Measure object at 0x75cdaf4170d0>, 'total.MEAN': <atoti.measure.Measure object at 0x75cdaf4171c0>, 'total.SUM': <atoti.measure.Measure object at 0x75cdaf4171f0>, 'anxiety.MEAN': <atoti.measure.Measure object at 0x75cdaf417160>, 'anxiety.SUM': <atoti.measure.Measure object at 0x75cdaf417190>, 'silver.MEAN': <atoti.measure.Measure object at 0x75cdaf417280>, 'silver.SUM': <atoti.measure.Measure object at 0x75cdaf4172b0>, 'gold.MEAN': <atoti.measure.Measure object at 0x75cdaf417220>, 'gold.SUM': <atoti.measure.Measure object at 0x75cdaf417250>, 'schizophrenia.MEAN': <atoti.measure.Measure object at 0x75cdaf417340>, 'schizophrenia.SUM': <atoti.measure.Measure object at 0x75cdaf417370>, 'eatingdisorder.MEAN': <atoti.measure.Measure object at 0x75cdaf4172e0>, 'eatingdisorder.SUM': <atoti.measure.Measure object at 0x75cdaf417310>}

In [33]:
testcube.query(measures["depression.SUM"],measures["total.SUM"], levels=[levels['Lifexpectancy', 'lifeexpectancy', 'population'],levels[('Countries', 'continent', 'country')]])

depression.SUM  \
lifeexpectancy population continent     country                         
High           High       Asia          Japan                2,594.42   
                                        Malaysia             1,122.02   
                          Europe        France               7,237.99   
                                        Germany              8,132.09   
                                        Italy                7,216.27   
                                        Netherlands          4,332.93   
                                        Poland               1,977.35   
                                        Romania              1,281.02   
                                        Spain                5,030.28   
                                        Ukraine                682.16   
                                        United Kingdom       5,919.54   
                          North America Canada               7,842.63   
                                        Mexico               1,223.57   
                                        United States       10,276.10   
                          Oceania       Australia            5,885.74   
                          South America Argentina            2,693.33   
                                        Brazil               2,998.17   
                                        Chile                  768.88   
               Low        Europe        Estonia                704.62   
                                        Latvia               2,593.75   
                                        Lithuania              705.96   
                                        Slovenia             2,406.68   
               Medium     Europe        Austria              4,362.95   
                                        Belarus              1,387.90   
                                        Belgium                569.82   
                                        Bulgaria               986.76   
                                        Croatia              1,612.49   
                                        Denmark              3,207.01   
                                        Finland              3,218.68   
                                        Greece               3,929.31   
                                        Hungary              2,960.70   
                                        Ireland                730.80   
                                        Lithuania              781.46   
                                        Norway               4,232.40   
                                        Portugal             1,723.48   
                                        Serbia                 456.12   
                                        Sweden               7,477.36   
                                        Switzerland          6,547.98   
                          Oceania       New Zealand          3,222.05   
Low            High       Africa        South Africa           670.11   
Medium         High       Africa        South Africa         1,449.38   
                          Asia          Indonesia            2,056.61   
                          Europe        Romania              1,295.85   
                                        Ukraine              3,899.84   
                          South America Brazil               1,551.98   
               Low        Europe        Estonia                838.08   
                                        Moldova                608.38   
               Medium     Europe        Belarus                724.33   
                                        Hungary                573.25   
N/A            N/A        Asia          China                4,845.83   
                                        North Korea            441.51   
                                        South Korea          2,366.61   
                          Europe        Russia               5,822.42   
                                        